In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as tts
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR
from sklearn.linear_model import PoissonRegressor as PR

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
boe=pd.read_csv('../data/BOE_def.csv')

In [6]:
boe.fecha_de_conclusion.unique()

array(['08-12-2022', '07-12-2022', '06-12-2022', ..., '06-02-2016',
       '27-01-2016', '10-01-2016'], dtype=object)

In [7]:
import regex as re

In [8]:
def limpiar_(fecha_de_inicio):
    fecha_de_inicio=re.findall('\d{4}', fecha_de_inicio)
    fecha_de_inicio=' '.join(fecha_de_inicio)
    return fecha_de_inicio

In [9]:
boe.fecha_de_inicio=boe.fecha_de_inicio.apply(limpiar_)

In [10]:
boe.fecha_de_conclusion=boe.fecha_de_conclusion.apply(limpiar_)

In [11]:
boe.head()

,identificador,tipo_de_subasta,cuenta_expediente,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,anuncio_BOE,valor_subasta,tasacion,puja_minima,tramos_entre_pujas,importe_del_deposito,direccion,codigo_postal,localidad,provincia,puja,pm2
0,SUB-JA-2022-204013,JUDICIAL EN VÍA DE APREMIO,4743 0000 05 1018 20,2022,2022,7145,BOE-B-2022-35270,108482,0,0,2169,5424,Urbanización Els Racons nº 5. partida Michafiga,46220,Picassent,Valencia/València,58580,1088
1,SUB-JA-2022-196067,JUDICIAL EN VÍA DE APREMIO,2368 0000 06 0191 20,2022,2022,142298,BOE-B-2022-35269,305832,0,0,6116,15291,"CALLE GABRIEL GARCÍA MÁRQUEZ, 31",28980,PARLA,Madrid,177382,1721
2,SUB-JA-2022-204060,JUDICIAL EN VÍA DE APREMIO,2995 0000 06 0456 16,2022,2022,55933,BOE-B-2022-35263,247902,0,0,0,12395,VIVIENDA 1 BLOQUE 19 CONJUNTO RESIDENCIAL HACI...,29600,Marbella,Málaga,252000,1721
3,SUB-JA-2022-198972,JUDICIAL EN VÍA DE APREMIO,3997 0000 06 1661 20,2022,2022,210425,BOE-B-2022-28879,826272,0,0,16525,41313,"AVENIDA DE JACARANDA, 5, URBANIZACION CLUB DE ...",41930,BORMUJOS,Sevilla,512288,1557
4,SUB-JA-2022-203383,JUDICIAL EN VÍA DE APREMIO,5418 0000 06 0169 21,2022,2022,34853,BOE-B-2022-35253,121684,121684,45978,2433,6084,"Calle Palma de Mallorca, nº 13, bloque 19, 3º ...",35016,Las Palmas de Gran Canaria,Las Palmas,45978,1130


In [12]:
boe.drop(columns=['identificador','tipo_de_subasta','cuenta_expediente',
                  'anuncio_BOE','tasacion','puja_minima','tramos_entre_pujas',
                 'importe_del_deposito','direccion','localidad'], inplace=True)

In [13]:
boe.head()

,fecha_de_inicio,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2
0,2022,2022,7145,108482,46220,Valencia/València,58580,1088
1,2022,2022,142298,305832,28980,Madrid,177382,1721
2,2022,2022,55933,247902,29600,Málaga,252000,1721
3,2022,2022,210425,826272,41930,Sevilla,512288,1557
4,2022,2022,34853,121684,35016,Las Palmas,45978,1130


In [14]:
boe.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6154 entries, 0 to 6153
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fecha_de_inicio      6154 non-null   object
 1   fecha_de_conclusion  6154 non-null   object
 2   cantidad_reclamada   6154 non-null   int64 
 3   valor_subasta        6154 non-null   int64 
 4   codigo_postal        6154 non-null   int64 
 5   provincia            6154 non-null   object
 6   puja                 6154 non-null   int64 
 7   pm2                  6154 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 1.4 MB


In [15]:
boe['fecha_de_inicio'] = boe['fecha_de_inicio'].astype(int)

In [16]:
boe['fecha_de_conclusion'] = boe['fecha_de_conclusion'].astype(int)

In [ ]:
#boe22=boe.loc[boe['fecha_de_conclusion'] == 2022]

In [17]:
#eliminamos esta columna porque el precio se cierra en la fecha de conclusion y varia segun el año final.
boe.drop(columns=['fecha_de_inicio'], inplace=True)

In [ ]:
#eliminamos codigo postal? con eso hemos sacado el pm2, no nos interesa más. Ademas contamos con la provincia.
boe.drop(columns=['codigo_postal'], inplace=True)

In [18]:
#creamos new colum con la diferencia entre la deuda(cantidad_reclamada) y el valor que le da el juez(valor_subasta)
boe['diference'] = boe['valor_subasta'].sub(boe['cantidad_reclamada'], axis = 0)

In [ ]:
boe.head()

In [ ]:
boe.drop(columns=['cantidad_reclamada','valor_subasta'], inplace=True)

In [ ]:
boe.head()

In [ ]:
boe.provincia.value_counts

In [19]:
def limpia_provincia(x):
    if 'Madrid' in x:
        return 'Madrid'
    elif 'Barcelona' in x:
        return 'Barcelona'
    elif 'Málaga' in x:
        return 'Málaga'
    elif 'Sevilla' in x:
        return 'Sevilla'
    elif 'Alicante' in x:
        return 'Alicante'
    elif 'Toledo' in x:
        return 'Toledo'
    elif 'Valencia/València' in x:
        return 'Valencia/València'
    elif 'Santa Cruz de Tenerife' in x:
        return 'Santa Cruz de Tenerife'
    elif 'Alicante/Alacant' in x:
        return 'Alicante/Alacant'
    elif 'Cádiz' in x:
        return 'Cádiz'
    elif 'Almería' in x:
        return 'Almería'
    elif 'Valencia' in x:
        return 'Valencia'
    elif 'Tarragona' in x:
        return 'Tarragona'
    elif 'Huelva' in x:
        return 'Huelva'
    elif 'Zaragoza' in x:
        return 'Zaragoza'
    elif 'Gerona' in x:
        return 'Gerona'
    elif 'Baleares' in x:
        return 'Baleares'
    elif 'Las Palmas' in x:
        return 'Las Palmas'
    elif 'Albacete' in x:
        return 'Albacete'
    elif 'Pontevedra' in x:
        return 'Pontevedra'
    elif 'Granada' in x:
        return 'Granada'
    elif 'Illes Balears' in x:
        return 'Illes Balears'
    elif 'Cantabria' in x:
        return 'Cantabria'
    elif 'Asturias' in x:
        return 'Asturias'
    elif 'Córdoba' in x:
        return 'Córdoba'
    else:
        return 'other'
    

In [20]:
boe.provincia=boe.provincia.apply(limpia_provincia)

In [21]:
boe.head()

,fecha_de_conclusion,cantidad_reclamada,valor_subasta,codigo_postal,provincia,puja,pm2,diference
0,2022,7145,108482,46220,Valencia/València,58580,1088,101337
1,2022,142298,305832,28980,Madrid,177382,1721,163534
2,2022,55933,247902,29600,Málaga,252000,1721,191969
3,2022,210425,826272,41930,Sevilla,512288,1557,615847
4,2022,34853,121684,35016,Las Palmas,45978,1130,86831


In [22]:
boe=pd.get_dummies(boe, columns=['fecha_de_conclusion','provincia','codigo_postal'], drop_first=True)

boe.head()

cantidad_reclamada  valor_subasta    puja   pm2  diference  \
0                7145         108482   58580  1088     101337   
1              142298         305832  177382  1721     163534   
2               55933         247902  252000  1721     191969   
3              210425         826272  512288  1557     615847   
4               34853         121684   45978  1130      86831   

   fecha_de_conclusion_2017  fecha_de_conclusion_2018  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2019  fecha_de_conclusion_2020  \
0                         0                         0   
1                         0                         0   
2                         0                         0   
3                         0                         0   
4                         0                         0   

   fecha_de_conclusion_2021  fecha_de_conclusion_2022  provincia_Alicante  \
0                         0                         1                   0   
1                         0                         1                   0   
2                         0                         1                   0   
3                         0                         1                   0   
4                         0                         1                   0   

   provincia_Almería  provincia_Asturias  provincia_Baleares  \
0                  0                   0                   0   
1                  0                   0                   0   
2                  0                   0                   0   
3                  0                   0                   0   
4                  0                   0                   0   

   provincia_Barcelona  provincia_Cantabria  provincia_Cádiz  \
0                    0                    0                0   
1                    0                    0                0   
2                    0                    0                0   
3                    0                    0                0   
4                    0                    0                0   

   provincia_Córdoba  provincia_Gerona  provincia_Granada  provincia_Huelva  \
0                  0                 0                  0                 0   
1                  0                 0                  0                 0   
2                  0                 0                  0                 0   
3                  0                 0                  0                 0   
4                  0                 0                  0                 0   

   provincia_Illes Balears  provincia_Las Palmas  provincia_Madrid  \
0                        0                     0                 0   
1                        0                     0                 1   
2                        0                     0                 0   
3                        0                     0                 0   
4                        0                     1                 0   

   provincia_Málaga  provincia_Pontevedra  provincia_Santa Cruz de Tenerife  \
0                 0                     0                                 0   
1                 0                     0                                 0   
2                 1                     0                                 0   
3                 0                     0                                 0   
4                 0                     0                                 0   

   provincia_Sevilla  provincia_Tarragona  provincia_Toledo  \
0                  0                    0                 0   
1                  0                    0                 0   
2                  0                    0                 0   
3                  1                    0                 0   
4      

In [ ]:
boe.info()

In [23]:
boe.cantidad_reclamada=LabelEncoder().fit_transform(boe.cantidad_reclamada)

In [24]:
boe.valor_subasta=LabelEncoder().fit_transform(boe.valor_subasta)

In [25]:
boe.pm2=LabelEncoder().fit_transform(boe.pm2)

In [26]:
boe.diference=LabelEncoder().fit_transform(boe.diference)

In [ ]:
boe.head()

In [ ]:
#boe.diference.unique()

In [ ]:
#boe=boe.rename(columns = {'diff':'diference'})

In [ ]:
boe.head()

In [27]:
boe_x=boe.drop('puja', axis=1)
boe_y=boe.puja

In [28]:
X_train, X_test, y_train, y_test = tts(boe_x, boe_y,train_size=0.9, test_size=0.1,random_state =71)

**Iniciamos los modelos**

In [29]:
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()
pr=PR()

**Entrenamos modelos**

In [30]:
linreg.fit(X_train, y_train)
lasso.fit(X_train, y_train)
ridge.fit(X_train, y_train)
elastic.fit(X_train, y_train)
svr.fit(X_train, y_train)
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)
pr.fit(X_train, y_train)

Learning rate set to 0.053657
0:	learn: 585003.0824095	total: 65.2ms	remaining: 1m 5s
1:	learn: 578564.9844845	total: 74.1ms	remaining: 37s
2:	learn: 572271.6423102	total: 80.9ms	remaining: 26.9s
3:	learn: 566123.2848662	total: 88.8ms	remaining: 22.1s
4:	learn: 560033.7793268	total: 100ms	remaining: 20s
5:	learn: 554020.9431253	total: 106ms	remaining: 17.6s
6:	learn: 547572.7214627	total: 115ms	remaining: 16.3s
7:	learn: 541718.0906931	total: 123ms	remaining: 15.3s
8:	learn: 536012.2469321	total: 133ms	remaining: 14.7s
9:	learn: 530120.7382953	total: 141ms	remaining: 13.9s
10:	learn: 524635.6678953	total: 147ms	remaining: 13.3s
11:	learn: 519136.6338131	total: 155ms	remaining: 12.8s
12:	learn: 514841.7985916	total: 163ms	remaining: 12.4s
13:	learn: 509534.8901098	total: 170ms	remaining: 12s
14:	learn: 504253.4495478	total: 176ms	remaining: 11.6s
15:	learn: 499141.7793712	total: 185ms	remaining: 11.4s
16:	learn: 492629.2917953	total: 193ms	remaining: 11.2s
17:	learn: 487387.2008279	tota

165:	learn: 140336.5748110	total: 1.5s	remaining: 7.55s
166:	learn: 138949.4738849	total: 1.51s	remaining: 7.53s
167:	learn: 138480.8691813	total: 1.51s	remaining: 7.51s
168:	learn: 137373.0462737	total: 1.53s	remaining: 7.5s
169:	learn: 136165.7842091	total: 1.54s	remaining: 7.5s
170:	learn: 134996.6742812	total: 1.54s	remaining: 7.48s
171:	learn: 133945.1143423	total: 1.55s	remaining: 7.46s
172:	learn: 132736.4490486	total: 1.56s	remaining: 7.45s
173:	learn: 131733.8932932	total: 1.57s	remaining: 7.44s
174:	learn: 130836.9303487	total: 1.57s	remaining: 7.41s
175:	learn: 129989.3187777	total: 1.58s	remaining: 7.4s
176:	learn: 129048.3087174	total: 1.59s	remaining: 7.39s
177:	learn: 128228.9682432	total: 1.6s	remaining: 7.37s
178:	learn: 127435.6463560	total: 1.61s	remaining: 7.37s
179:	learn: 126548.0096141	total: 1.62s	remaining: 7.37s
180:	learn: 125754.7100643	total: 1.63s	remaining: 7.35s
181:	learn: 124685.0390242	total: 1.63s	remaining: 7.34s
182:	learn: 123791.0207425	total: 1.

317:	learn: 79238.5395729	total: 2.64s	remaining: 5.65s
318:	learn: 79198.3716672	total: 2.64s	remaining: 5.64s
319:	learn: 79117.9094149	total: 2.65s	remaining: 5.64s
320:	learn: 79039.5721268	total: 2.66s	remaining: 5.63s
321:	learn: 78776.8552333	total: 2.67s	remaining: 5.62s
322:	learn: 78700.3378694	total: 2.68s	remaining: 5.62s
323:	learn: 78549.4821397	total: 2.69s	remaining: 5.61s
324:	learn: 78242.2531936	total: 2.69s	remaining: 5.6s
325:	learn: 78168.1088770	total: 2.7s	remaining: 5.58s
326:	learn: 78129.6504456	total: 2.71s	remaining: 5.58s
327:	learn: 78056.8327645	total: 2.72s	remaining: 5.57s
328:	learn: 78019.5738958	total: 2.73s	remaining: 5.56s
329:	learn: 77948.6400597	total: 2.73s	remaining: 5.55s
330:	learn: 77690.5399066	total: 2.74s	remaining: 5.54s
331:	learn: 77620.4672907	total: 2.75s	remaining: 5.54s
332:	learn: 77307.8113561	total: 2.76s	remaining: 5.53s
333:	learn: 77272.6344937	total: 2.77s	remaining: 5.52s
334:	learn: 77197.2254082	total: 2.78s	remaining: 

468:	learn: 66691.7657070	total: 3.76s	remaining: 4.26s
469:	learn: 66674.6374756	total: 3.76s	remaining: 4.24s
470:	learn: 66576.6069996	total: 3.77s	remaining: 4.24s
471:	learn: 66540.0215602	total: 3.78s	remaining: 4.23s
472:	learn: 66451.2989135	total: 3.79s	remaining: 4.22s
473:	learn: 66409.0160912	total: 3.79s	remaining: 4.21s
474:	learn: 66293.9786073	total: 3.8s	remaining: 4.2s
475:	learn: 66258.0989657	total: 3.81s	remaining: 4.19s
476:	learn: 66241.2692458	total: 3.81s	remaining: 4.18s
477:	learn: 66204.2124054	total: 3.82s	remaining: 4.17s
478:	learn: 66187.7527206	total: 3.82s	remaining: 4.16s
479:	learn: 66147.0047361	total: 3.83s	remaining: 4.15s
480:	learn: 66108.7446237	total: 3.83s	remaining: 4.14s
481:	learn: 66073.8604928	total: 3.84s	remaining: 4.13s
482:	learn: 65961.0349870	total: 3.85s	remaining: 4.12s
483:	learn: 65858.6583665	total: 3.86s	remaining: 4.11s
484:	learn: 65721.1452469	total: 3.86s	remaining: 4.1s
485:	learn: 65704.9917138	total: 3.87s	remaining: 4

617:	learn: 60989.0033341	total: 4.83s	remaining: 2.98s
618:	learn: 60969.1143013	total: 4.84s	remaining: 2.98s
619:	learn: 60949.7874615	total: 4.84s	remaining: 2.97s
620:	learn: 60874.5287449	total: 4.86s	remaining: 2.96s
621:	learn: 60852.0025422	total: 4.86s	remaining: 2.96s
622:	learn: 60840.8797222	total: 4.87s	remaining: 2.95s
623:	learn: 60821.9792569	total: 4.89s	remaining: 2.94s
624:	learn: 60800.8998858	total: 4.89s	remaining: 2.94s
625:	learn: 60728.9535204	total: 4.9s	remaining: 2.93s
626:	learn: 60634.3525131	total: 4.91s	remaining: 2.92s
627:	learn: 60623.2670477	total: 4.92s	remaining: 2.91s
628:	learn: 60604.8276394	total: 4.93s	remaining: 2.9s
629:	learn: 60594.3468076	total: 4.93s	remaining: 2.9s
630:	learn: 60576.4269206	total: 4.94s	remaining: 2.89s
631:	learn: 60565.5654917	total: 4.95s	remaining: 2.88s
632:	learn: 60543.6156151	total: 4.96s	remaining: 2.87s
633:	learn: 60473.6921721	total: 4.96s	remaining: 2.87s
634:	learn: 60462.9045614	total: 4.97s	remaining: 2

781:	learn: 57084.7057772	total: 5.98s	remaining: 1.67s
782:	learn: 57068.2225498	total: 5.98s	remaining: 1.66s
783:	learn: 57060.3434911	total: 5.99s	remaining: 1.65s
784:	learn: 56973.8466233	total: 6s	remaining: 1.64s
785:	learn: 56965.9723528	total: 6s	remaining: 1.63s
786:	learn: 56958.1639016	total: 6.01s	remaining: 1.63s
787:	learn: 56908.4204400	total: 6.01s	remaining: 1.62s
788:	learn: 56890.2448658	total: 6.02s	remaining: 1.61s
789:	learn: 56855.9857482	total: 6.03s	remaining: 1.6s
790:	learn: 56840.3544344	total: 6.03s	remaining: 1.59s
791:	learn: 56832.9399483	total: 6.04s	remaining: 1.58s
792:	learn: 56803.1493383	total: 6.04s	remaining: 1.58s
793:	learn: 56747.1812668	total: 6.05s	remaining: 1.57s
794:	learn: 56739.4142230	total: 6.06s	remaining: 1.56s
795:	learn: 56731.7368879	total: 6.06s	remaining: 1.55s
796:	learn: 56717.3756048	total: 6.07s	remaining: 1.55s
797:	learn: 56655.1130446	total: 6.08s	remaining: 1.54s
798:	learn: 56647.4558491	total: 6.08s	remaining: 1.53s

935:	learn: 54213.4848191	total: 6.94s	remaining: 474ms
936:	learn: 54201.1494235	total: 6.95s	remaining: 467ms
937:	learn: 54195.1636943	total: 6.95s	remaining: 460ms
938:	learn: 54166.6781260	total: 6.96s	remaining: 452ms
939:	learn: 54114.2985427	total: 6.97s	remaining: 445ms
940:	learn: 54105.8459549	total: 6.97s	remaining: 437ms
941:	learn: 54092.7008419	total: 6.98s	remaining: 430ms
942:	learn: 54086.7470932	total: 6.99s	remaining: 422ms
943:	learn: 54081.0218265	total: 6.99s	remaining: 415ms
944:	learn: 54055.4542144	total: 7s	remaining: 407ms
945:	learn: 54028.7394866	total: 7s	remaining: 400ms
946:	learn: 54001.9480968	total: 7.01s	remaining: 392ms
947:	learn: 53995.7069789	total: 7.01s	remaining: 385ms
948:	learn: 53986.8434009	total: 7.02s	remaining: 377ms
949:	learn: 53980.9615392	total: 7.03s	remaining: 370ms
950:	learn: 53975.4160121	total: 7.03s	remaining: 362ms
951:	learn: 53959.4796329	total: 7.04s	remaining: 355ms
952:	learn: 53954.0021900	total: 7.04s	remaining: 347m

PoissonRegressor()

In [31]:
from sklearn.metrics import mean_squared_error as mse

In [32]:
from sklearn.metrics import r2_score as r2

In [33]:
#Import Lazypredict and all libraries
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np

In [34]:
# regresion lineal

y_pred=linreg.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(711193.583094276, -1.0745070820881737)

In [35]:
linreg.score(X_train, y_train), linreg.score(X_test, y_test)


(0.35689599209198475, -1.0745070820881737)

In [36]:
# lasso

y_pred=lasso.predict(X_test)

mse(y_test, y_pred, squared=False) , r2(y_test, y_pred)

(711167.1266691538, -1.0743527414930298)

In [37]:
lasso.score(X_train, y_train), lasso.score(X_test, y_test)

(0.3568708561055657, -1.0743527414930298)

In [38]:
# ridge

y_pred=ridge.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(619076.9156584926, -0.5719129189861776)

In [39]:
# elastic

y_pred=elastic.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)

(473612.75659962907, 0.08000377404898473)

In [40]:
## rfr

y_pred=rfr.predict(X_test)

mse(y_test, y_pred, squared=False)  , r2(y_test, y_pred)# RMSE

(185534.30774874403, 0.8588152727331123)

In [41]:
##gbr
y_pred=gbr.predict(X_test)

mse(y_test, y_pred, squared=False) , r2(y_test, y_pred) 

(318114.4844199259, 0.5849445215127627)

In [42]:
##xgbr

y_pred=xgbr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(1135186.4784607964, -4.285349146879067)

In [43]:
##ctr
y_pred=ctr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(379079.5016365287, 0.4106139331421296)

In [44]:
## lgbmr
y_pred=lgbmr.predict(X_test)

mse(y_test, y_pred, squared=False), r2(y_test, y_pred)  # RMSE

(180687.45877194346, 0.8660954659182787)

In [45]:
#lgbmr 40%r2 aplicando funcion provincias, el resto dummies y labelencoder. 9 1 71

In [46]:
lazy = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = lazy.fit(X_train, X_test, y_train, y_test)

100%|████████████████████████████████████████| 42/42 [1:25:21<00:00, 121.94s/it]


In [47]:
print(models)

                                         Adjusted R-Squared  \
Model                                                         
RANSACRegressor               86889385068836295869988864.00   
TransformedTargetRegressor    44943274185148258112765952.00   
LinearRegression              44943274185148258112765952.00   
SGDRegressor                                      520984.97   
XGBRegressor                                           3.13   
KernelRidge                                            1.87   
OrthogonalMatchingPursuitCV                            1.84   
OrthogonalMatchingPursuit                              1.84   
LassoLars                                              1.84   
Lars                                                   1.84   
Ridge                                                  1.84   
Lasso                                                  1.84   
RidgeCV                                                1.83   
LassoLarsIC                                            

In [ ]:
boe_sinPuja=pd.read_csv('../data/BOE_def.csv')

boe_sinPuja.info()

In [ ]:
boe_sinPuja.drop(columns=['puja'], inplace=True)

In [ ]:
boe_sinPuja.head()

In [ ]:
boe_sinPuja.to_csv('../data/BOE_Sin_Puja.csv', index=False, sep=',') 

In [ ]:
pred_lgbmr=lgbmr.predict(boe)